In [1]:
import numpy as np
import sys
from casadi import *

# Add do_mpc to path. This is not necessary if it was installed via pip
sys.path.append('../../../')

# Import do_mpc package:
import do_mpc

In [2]:
a = horzcat(DM([1,2]),DM([3,4]),DM([5,6]),DM([7,8]),DM([9,10])  )
print(a)
# a = reshape(a,5,2)
a = a.T
print(a)

print(a[2,:])


[[1, 3, 5, 7, 9], 
 [2, 4, 6, 8, 10]]

[[1, 2], 
 [3, 4], 
 [5, 6], 
 [7, 8], 
 [9, 10]]
[[5, 6]]


In [3]:
print(DM([0,-9.8]).shape)
print((DM([0,-9.8]).T).shape)

(2, 1)
(1, 2)


In [4]:

model = do_mpc.model.Model('continuous', 'MX')

In [5]:
g = 9.80665 # m/s^2, Gravity
N = 10

In [6]:
# Add the velocity vector to each of the position vectors in the rope
test_rope = DM([[2,1],[5,5],[6,6]])
test_velocity = DM([8,9])
# print(test_rope + repmat(test_velocity.T,N,1))

In [7]:
m = MX.sym('m',N,2)
y1 = MX.sym('y1',2)
y2 = MX.sym('y2',2)

# f = Function('f',[m,y1],\
#            [m + repmat(y1.T,N,1)])

# f( test_rope  ,  test_velocity  )

In [8]:

# force applied to vec1
# ie force that moves vec1 towards vec2

# Fs=-k*(mag(L)-L0)*norm(L)

vec1 = MX.sym('vec1',2)
vec2 = MX.sym('vec2',2)




# vector length

v_len = Function('f',[vec1],\
           [ sqrt(vec1[0]**2 + vec1[1]**2 + 0.00001)  ])

# add 0.00001 so you don't divide by 0

v_norm = Function('f',[vec1],\
           [ reshape((vec1 / (v_len(vec1) + 0.00001)),1,2)  ])

print(v_len(DM([3,4])))
print(v_norm(DM([3,4])))


5
[[0.599999, 0.799998]]


In [9]:
# Fs=-k*(mag(L)-L0)*norm(L)
k=5
l=5
F_spring = Function('f',[vec1,vec2],\
           [ -k*( v_len(vec1-vec2)  - l) * v_norm(vec1-vec2) ])






In [10]:



pos = model.set_variable('_x',  'pos',shape=(N,2))
dpos = model.set_variable('_x',  'dpos',shape=(N,2))

print(model.x.labels())
u = model.set_variable('_u',  'u',shape=(1,2))
print(u.shape)

# ddpos = model.set_variable('_z', 'ddpos',shape=(N,2))

['[pos,0]', '[pos,1]', '[pos,2]', '[pos,3]', '[pos,4]', '[pos,5]', '[pos,6]', '[pos,7]', '[pos,8]', '[pos,9]', '[pos,10]', '[pos,11]', '[pos,12]', '[pos,13]', '[pos,14]', '[pos,15]', '[pos,16]', '[pos,17]', '[pos,18]', '[pos,19]', '[dpos,0]', '[dpos,1]', '[dpos,2]', '[dpos,3]', '[dpos,4]', '[dpos,5]', '[dpos,6]', '[dpos,7]', '[dpos,8]', '[dpos,9]', '[dpos,10]', '[dpos,11]', '[dpos,12]', '[dpos,13]', '[dpos,14]', '[dpos,15]', '[dpos,16]', '[dpos,17]', '[dpos,18]', '[dpos,19]']
(1, 2)


In [11]:
model.set_rhs('pos', dpos)


# if you don't add the [0,:] then it indexes the single number 
# ie pos[0] = the x value of pos[0,:]
# ie pos[2] = the x value of pos[1,:]
print(pos[0,:].shape)
print(v_norm(pos[0,:]).shape)
ddpos = vertcat(
    u - pos[0],
    F_spring(pos[1,:],pos[0,:]) + F_spring(pos[1,:],pos[2,:]),
    F_spring(pos[2,:],pos[1,:]) + F_spring(pos[2,:],pos[3,:]),
    F_spring(pos[3,:],pos[2,:]) + F_spring(pos[3,:],pos[4,:]),
    F_spring(pos[4,:],pos[3,:]) + F_spring(pos[4,:],pos[5,:]),
    F_spring(pos[5,:],pos[4,:]) + F_spring(pos[5,:],pos[6,:]),
    F_spring(pos[6,:],pos[5,:]) + F_spring(pos[6,:],pos[7,:]),
    F_spring(pos[7,:],pos[6,:]) + F_spring(pos[7,:],pos[8,:]),
    F_spring(pos[8,:],pos[7,:]) + F_spring(pos[8,:],pos[9,:]),
    F_spring(pos[9,:],pos[8,:]) + DM([0,-9.8]).T
)
print(ddpos.shape)
print(dpos.shape)
print(pos.shape)
print(u.shape)

model.set_rhs('dpos', ddpos)
# model.set_rhs('time', 1)

(1, 2)
(1, 2)
(10, 2)
(10, 2)
(10, 2)
(1, 2)


In [12]:
# formulas = vertcat(
#         # 1
#         u
#         - (ddpos),
#     )

# model.set_alg('formulas', formulas)

In [13]:

distance = v_len(DM([50,50]).T-pos[N-1,:])


model.set_expression('distance', distance)

MX(f((all_50(1x2)-pos[9:29:10])'){0})

In [14]:
# Build the model
model.setup()

In [15]:
mpc = do_mpc.controller.MPC(model)

In [16]:
setup_mpc = {
    'n_horizon': 100,
    'n_robust': 0,
    'open_loop': 0,
    't_step': 0.04,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 3,
    'collocation_ni': 1,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.linear_solver': 'mumps'}
}
mpc.set_param(**setup_mpc)

In [17]:
mterm = model.aux['distance'] # terminal cost
lterm = model.aux['distance'] # stage cost

mpc.set_objective(mterm=mterm, lterm=lterm)
# Input force is implicitly restricted through the objective.
mpc.set_rterm(u=0.1)

In [18]:
mpc.bounds['lower','_u','u'] = DM([0,0]).T
mpc.bounds['upper','_u','u'] = DM([100,100]).T

In [19]:
mpc.setup()

RuntimeError: .../casadi/core/sparsity.cpp:1358: Assertion "sp_nrow==ret_nrow || sp_nrow==0" failed:
Sparsity::horzcat: Mismatching number of rows

In [ ]:
estimator = do_mpc.estimator.StateFeedback(model)

In [ ]:
simulator = do_mpc.simulator.Simulator(model)

In [ ]:
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 0.04
}

simulator.set_param(**params_simulator)

In [ ]:
simulator.setup()

In [ ]:
simulator.x0['pos'] = repmat(DM([10,10]).T,N,1)
simulator.x0['dpos'] = repmat(DM([0,0]).T,N,1)

x0 = simulator.x0.cat.full()

mpc.x0 = x0
estimator.x0 = x0

mpc.set_initial_guess()


In [ ]:
import matplotlib.pyplot as plt
plt.ion()
from matplotlib import rcParams
rcParams['text.usetex'] = False
rcParams['axes.grid'] = True
rcParams['lines.linewidth'] = 2.0
rcParams['axes.labelsize'] = 'xx-large'
rcParams['xtick.labelsize'] = 'xx-large'
rcParams['ytick.labelsize'] = 'xx-large'

In [ ]:
mpc_graphics = do_mpc.graphics.Graphics(mpc.data)

In [ ]:
%%capture

fig = plt.figure(figsize=(16,9))


ax2 = plt.subplot2grid((4, 2), (0, 1))

ax4 = plt.subplot2grid((4, 2), (2, 1))
ax5 = plt.subplot2grid((4, 2), (3, 1))

ax2.set_ylabel('distance')

ax4.set_ylabel('Pos ')
ax5.set_ylabel('U')

# Axis on the right.
for ax in [ax2, ax4, ax5]:
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    if ax != ax5:
        ax.xaxis.set_ticklabels([])

ax5.set_xlabel('time [s]')

mpc_graphics.add_line(var_type='_aux', var_name='distance', axis=ax2)

mpc_graphics.add_line(var_type='_x', var_name='pos', axis=ax4)
mpc_graphics.add_line(var_type='_u', var_name='u', axis=ax5)


fig.align_ylabels()
fig.tight_layout()

In [ ]:
u0 = mpc.make_step(x0)

In [ ]:


mpc_graphics.plot_predictions()
mpc_graphics.reset_axes()

fig

In [ ]:
%%capture
# Quickly reset the history of the MPC data object.
mpc.reset_history()

n_steps = 100
for k in range(n_steps):
    u0 = mpc.make_step(x0)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)

In [ ]:
from matplotlib.animation import FuncAnimation, FFMpegWriter, ImageMagickWriter

# The function describing the gif:
x_arr = mpc.data['_x']
def update(t_ind):
#     line1, line2 = pendulum_bars(x_arr[t_ind])
#     bar1[0].set_data(line1[0],line1[1])
#     bar2[0].set_data(line2[0],line2[1])
    mpc_graphics.plot_results(t_ind)
    mpc_graphics.plot_predictions(t_ind)
    mpc_graphics.reset_axes()


anim = FuncAnimation(fig, update, frames=n_steps, repeat=False)
gif_writer = ImageMagickWriter(fps=20)
anim.save('anim.gif', writer=gif_writer)

In [ ]:
from IPython.display import HTML, Image
display(Image(data=open("anim.gif",'rb').read(), format='png'))